In [1]:
from googleapiclient.discovery import build
import nltk
import numpy as np
import pandas as pd
import time
# https://github.com/googleapis/google-api-python-client/blob/master/googleapiclient/discovery.py

In [2]:
from googlesearch import search

def google_search(query):
    return search(query, lang = 'en',  # The language
                num = 10, start = 0, stop = None, pause = 2.0)


In [10]:
def sim(P, Q, threshold):
    
    print('Extracting P...')
    results_p = google_search(P)
    count_p = len(list(results_p))
    
    time.sleep(10)
    print('Extracting Q...')
    results_q = google_search(Q)
    count_q = len(list(results_q))
    
    time.sleep(40)
    
    print('Extracting P AND Q...')
    results_pq = google_search('{} AND {}'.format(P,Q))
    count_pq = len(list(results_pq))
    
    
    if count_pq <= threshold:
        return 0
    
    else:
        return (count_pq) /(count_p + count_q + count_pq) 
        
        
    

In [11]:
threshold, P, Q = 5, 'Iphone', 'Huawei'
sim(P, Q, threshold)

Extracting P...
Extracting Q...
Extracting P AND Q...


0.4